<a href="https://colab.research.google.com/github/B-MEbrahim/Torch/blob/main/DLwPT/ch_6/dlwpt_ch6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim

torch.set_printoptions(edgeitems=2, linewidth=75)

In [2]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

t_u.shape

torch.Size([11, 1])

In [3]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([ 7,  9,  3,  5, 10,  6,  0,  8,  1]), tensor([2, 4]))

In [4]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

In [5]:
import torch.nn as nn

linear_model = nn.Linear(1, 1)
linear_model(t_un_val)

tensor([[1.5858],
        [1.5084]], grad_fn=<AddmmBackward0>)

In [6]:
linear_model.weight

Parameter containing:
tensor([[0.4074]], requires_grad=True)

In [7]:
linear_model.bias

Parameter containing:
tensor([-0.7855], requires_grad=True)

In [8]:
x = torch.ones(1)
linear_model(x)

tensor([-0.3780], grad_fn=<ViewBackward0>)

In [11]:
x = torch.ones(10, 1)
linear_model(x)

tensor([[-0.3780],
        [-0.3780],
        [-0.3780],
        [-0.3780],
        [-0.3780],
        [-0.3780],
        [-0.3780],
        [-0.3780],
        [-0.3780],
        [-0.3780]], grad_fn=<AddmmBackward0>)

In [12]:
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(
    linear_model.parameters(),
    lr=1e-2
)

In [13]:
linear_model.parameters

<bound method Module.parameters of Linear(in_features=1, out_features=1, bias=True)>

In [14]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[0.3962]], requires_grad=True),
 Parameter containing:
 tensor([-0.4467], requires_grad=True)]

In [15]:
def training_loop(n_epochs, optimizer, model, loss_fn,
                  t_u_train, t_u_val,
                  t_c_train, t_c_val):
  for epoch in range(1, n_epochs + 1):
    t_p_train = model(t_u_train)
    loss_train = loss_fn(t_p_train, t_c_train)

    t_p_val = model(t_u_val)
    loss_val = loss_fn(t_p_val, t_c_val)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    if epoch == 1 or epoch % 1000 == 0:
      print(f"Epoch {epoch}, Trainging loss {loss_train}:.4f",
            f"Validation loss {loss_val}:.4f")


In [16]:
def loss_fn(t_p, t_c):
  squared_diff = (t_p - t_c) ** 2
  return squared_diff.mean()

linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(linear_model.parameters(), lr=1e-2)

training_loop(
    n_epochs=3000,
    optimizer=optimizer,
    model=linear_model,
    loss_fn=loss_fn,
    t_u_train=t_un_train,
    t_u_val=t_un_val,
    t_c_train=t_c_train,
    t_c_val=t_c_val
)
print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Trainging loss 185.51113891601562:.4f Validation loss 166.30963134765625:.4f
Epoch 1000, Trainging loss 3.5776007175445557:.4f Validation loss 2.5763092041015625:.4f
Epoch 2000, Trainging loss 3.043879508972168:.4f Validation loss 2.504833936691284:.4f
Epoch 3000, Trainging loss 3.0354928970336914:.4f Validation loss 2.4961354732513428:.4f

Parameter containing:
tensor([[5.3717]], requires_grad=True)
Parameter containing:
tensor([-17.2269], requires_grad=True)


In [17]:
seq_model = nn.Sequential(
    nn.Linear(1, 13),
    nn.Tanh(),
    nn.Linear(13, 1)
)
seq_model

Sequential(
  (0): Linear(in_features=1, out_features=13, bias=True)
  (1): Tanh()
  (2): Linear(in_features=13, out_features=1, bias=True)
)

In [19]:
for name, param in seq_model.named_parameters():
  print(name, param.shape)

0.weight torch.Size([13, 1])
0.bias torch.Size([13])
2.weight torch.Size([1, 13])
2.bias torch.Size([1])


In [21]:
from collections import OrderedDict

seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(1, 8)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(8, 1))
]))
seq_model

Sequential(
  (hidden_linear): Linear(in_features=1, out_features=8, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=8, out_features=1, bias=True)
)

In [22]:
for name, param in seq_model.named_parameters():
    print(name, param.shape)

hidden_linear.weight torch.Size([8, 1])
hidden_linear.bias torch.Size([8])
output_linear.weight torch.Size([1, 8])
output_linear.bias torch.Size([1])


In [23]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-3) # <1>

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val,
    t_c_train = t_c_train,
    t_c_val = t_c_val)

print('output', seq_model(t_un_val))
print('answer', t_c_val)
print('hidden', seq_model.hidden_linear.weight.grad)

Epoch 1, Trainging loss 176.8362579345703:.4f Validation loss 154.4783935546875:.4f
Epoch 1000, Trainging loss 4.506984233856201:.4f Validation loss 2.368572235107422:.4f
Epoch 2000, Trainging loss 2.3652288913726807:.4f Validation loss 1.837519645690918:.4f
Epoch 3000, Trainging loss 2.14286470413208:.4f Validation loss 2.118964195251465:.4f
Epoch 4000, Trainging loss 1.9180965423583984:.4f Validation loss 2.0727312564849854:.4f
Epoch 5000, Trainging loss 1.8419020175933838:.4f Validation loss 2.09771990776062:.4f
output tensor([[13.4970],
        [12.2204]], grad_fn=<AddmmBackward0>)
answer tensor([[15.],
        [11.]])
hidden tensor([[-1.4214e-03],
        [ 1.3251e+00],
        [-3.9169e+00],
        [-1.2754e-02],
        [ 4.2699e-01],
        [ 4.3675e+00],
        [ 2.0510e-01],
        [ 4.4466e+00]])
